In [389]:
import h5py
import numpy as np
import pandas as pd
from astropy.io import ascii
from gc_utils import iteration_name, snapshot_name  # type: ignore

In [390]:
simulation = "m12i"
offset = 4
it = 1
# snapshot = 356
snapshot = 592
sim_dir = "/Users/z5114326/Documents/simulations/"
data_dir = "/Users/z5114326/Documents/GitHub/gc_process_katana/data/"

In [391]:
proc_file = sim_dir + simulation + "/" + simulation + "_processed.hdf5"
proc_data = h5py.File(proc_file, "a")  # open processed data file

In [392]:
raw_dir = data_dir + "results/" + simulation + "/raw/it_%d/" % it
it_id = iteration_name(it)

gc_id = proc_data[it_id]["source"]["gc_id"]
analyse_flag = proc_data[it_id]["source"]["analyse_flag"]
group_id = proc_data[it_id]["source"]["group_id"]

snap_zform = proc_data[it_id]["source"]["snap_zform"]
snap_last_lst = proc_data[it_id]["source"]["last_snap"]
snap_accr = proc_data[it_id]["source"]["snap_acc"]

ptypes_byte = proc_data[it_id]["source"]["ptype"]
ptypes = [ptype.decode("utf-8") for ptype in ptypes_byte]

In [393]:
public_snapshot_fil = data_dir + "external/snapshot_times_public.txt"
with open(public_snapshot_fil) as f:
    content = f.readlines()
    content = content[13:]
snap_pub = ascii.read(content)

t_form_lst = proc_data[it_id]["source"]["form_time"]
t_dis_lst = proc_data[it_id]["source"]["t_dis"]

gc_id_snap = []
group_id_snap = []
ptype_snap = []
snap_accr_snap = []

idx = np.where(snap_pub["index"] == snapshot)[0][0]
time = snap_pub["time[Gyr]"][idx]
snap_id = snapshot_name(snapshot)

for gc, group, ptype, a_flag, t_form, t_dis, snap_ac in zip(
    gc_id, group_id, ptypes, analyse_flag, t_form_lst, t_dis_lst, snap_accr
):
    if a_flag == 0:
        continue

    if time < t_form:
        continue

    if (t_dis != -1) and (t_dis < time):
        continue

    gc_id_snap.append(gc)
    group_id_snap.append(group)
    ptype_snap.append(ptype)
    snap_accr_snap.append(snap_ac)

In [394]:
gc_id_snap = []
group_id_snap = []
ptype_snap = []
snap_accr_snap = []

snap_id = snapshot_name(snapshot)
for gc, group, ptype, a_flag, snap_form, snap_last, snap_ac, t_dis in zip(
    gc_id, group_id, ptypes, analyse_flag, snap_zform, snap_last_lst, snap_accr, t_dis_lst
):
    if a_flag == 0:
        continue

    if snapshot < snap_form:
        continue

    if (t_dis != -1) and (snap_last <= snapshot):
        continue

    gc_id_snap.append(gc)
    group_id_snap.append(group)
    ptype_snap.append(ptype)
    snap_accr_snap.append(snap_ac)

In [395]:
np.max(snap_last_lst)

np.int64(600)

In [396]:
snap_mass_file = raw_dir + "allcat_s-%d_p2-7_p3-1_k-1.5_logm_snap%d.txt" % (it, snapshot - offset)
gc_id_file = raw_dir + "allcat_s-%d_p2-7_p3-1_gcid.txt" % it


gcid_df = pd.read_csv(gc_id_file, sep=" ").drop(["ID"], axis=1)
gcid_df.columns = ["GC_ID", "quality"]

mass_snap_df = pd.read_csv(snap_mass_file, header=None)
mass_snap_df.columns = ["mass"]

comb_df = pd.concat([gcid_df, mass_snap_df], axis=1)

In [397]:
filt_comb_df = comb_df[comb_df["mass"] != -1]
filt_comb_df

,GC_ID,quality,mass
144,57057454,2,4.926
171,50061388,2,4.329
182,57058931,2,4.197
207,64439473,2,4.528
350,119804751,0,4.209
...,...,...,...
12031,50308553,1,5.258
12107,47819237,2,5.262
12148,60013501,1,4.752
12177,47435620,2,4.425


In [398]:
indexes = filt_comb_df[filt_comb_df["GC_ID"].isin(gc_id_snap)].index
indexes

Index([  144,   171,   182,   207,   350,   369,   370,   397,   559,  1145,
       ...
       11694, 11869, 11934, 11953, 11981, 12031, 12107, 12148, 12177, 12201],
      dtype='int64', length=228)

In [399]:
print(len(gc_id_snap), len(filt_comb_df))

228 228


In [400]:
test_list = [gc for gc in filt_comb_df["GC_ID"]]

In [401]:
print(len(gc_id_snap), len(test_list))

228 228


In [402]:
for gc in test_list:
    if gc not in gc_id_snap:
        print(gc)

In [403]:
for gc in gc_id_snap:
    if gc not in test_list:
        print(gc)

In [405]:
proc_data.close()